**<h1 align="center">CSV Preprocessing</h1>**

## Global Variables

### Project Specific Variables

In [ ]:
# CSV Files
CSV_FOLDER = "../../data/"
# CSV_FOLDER = "/home/pyuser/data/"
CSV_LABELS_FILE = "paradise_csi.csv"
CSV_PATIENTS_FILE = "PatientIds.csv"
CSV_ARCHIMED_DATA = "ArchiMed_Data.csv"
CSV_TO_EXPLORE_1 = "paradise_csi_drop_non_nan_w_classes.csv"
CSV_TO_EXPLORE_2 = "paradise_csi_w_classes_w_non_nan.csv"
CSV_SEPARATOR = ","  # Specify the CSV separator, e.g., ',' or '\t'
IMPORT_COLUMNS = []  # If empty, import all columns
CHUNK_SIZE = 50000  # Number of rows per chunk

# Project Specific Variables
EXAM_CODE_START = "2020-128 01-"


### Colors

In [ ]:
# ANSI escape codes for colored output
ANSI = {
    'R' : '\033[91m',  # Red
    'G' : '\033[92m',  # Green
    'B' : '\033[94m',  # Blue
    'Y' : '\033[93m',  # Yellow
    'W' : '\033[0m',  # White
}

## Import CSVs to Dataframe

### CSV Import

In [ ]:
import pandas as pd
import numpy as np

# Import CSV files into dataframes
try:
    # Import labels data
    df_labels = pd.read_csv(
        CSV_FOLDER + CSV_LABELS_FILE,
        sep=CSV_SEPARATOR,
        usecols=IMPORT_COLUMNS if IMPORT_COLUMNS else None,
        chunksize=CHUNK_SIZE
    )
    df_labels = pd.concat(df_labels, ignore_index=True)
    print(f"{ANSI['G']}Successfully imported{ANSI['W']} {CSV_LABELS_FILE}")
    
    # Import patient data  
    # df_patients = pd.read_csv(
    #     CSV_FOLDER + CSV_PATIENTS_FILE,
    #     sep=CSV_SEPARATOR,
    #     usecols=IMPORT_COLUMNS if IMPORT_COLUMNS else None,
    #     chunksize=CHUNK_SIZE
    # )
    # df_patients = pd.concat(df_patients, ignore_index=True)
    # print(f"{ANSI['G']}Successfully imported{ANSI['W']} {CSV_PATIENTS_FILE}")
    
    # Import ArchiMed CSV
    df_archimed = pd.read_csv(
        CSV_FOLDER + CSV_ARCHIMED_DATA,
        sep=';',  # ArchiMed CSV separator is ';'
        usecols=IMPORT_COLUMNS if IMPORT_COLUMNS else None,
        chunksize=CHUNK_SIZE
    )
    df_archimed = pd.concat(df_archimed, ignore_index=True)
    print(f"{ANSI['G']}Successfully imported{ANSI['W']} {CSV_ARCHIMED_DATA}")
    
    # Import CSV to explore 1
    # df_to_explore_1 = pd.read_csv(
    #     CSV_FOLDER + CSV_TO_EXPLORE_1,
    #     sep=CSV_SEPARATOR,
    #     usecols=IMPORT_COLUMNS if IMPORT_COLUMNS else None,
    #     chunksize=CHUNK_SIZE
    # )
    # df_to_explore_1 = pd.concat(df_to_explore_1, ignore_index=True)
    # print(f"{ANSI['G']}Successfully imported{ANSI['W']} {CSV_TO_EXPLORE_1}")
    
    # Import CSV to explore 2
    # df_to_explore_2 = pd.read_csv(
    #     CSV_FOLDER + CSV_TO_EXPLORE_2,
    #     sep=CSV_SEPARATOR,
    #     usecols=IMPORT_COLUMNS if IMPORT_COLUMNS else None,
    #     chunksize=CHUNK_SIZE
    # )
    # df_to_explore_2 = pd.concat(df_to_explore_2, ignore_index=True)
    # print(f"{ANSI['G']}Successfully imported{ANSI['W']} {CSV_TO_EXPLORE_2}")

except Exception as e:
    print(f"{ANSI['R']}Error importing CSV files: {str(e)}{ANSI['W']}")


## Rename columns

### ArchiMed CSV

In [ ]:
# Rename columns in df_archimed to be more readable
df_archimed.rename(columns={
    "Exam": 'ExamCode',  # Not really the patient's name but the exam code
    'Instance Number': 'InstanceNumber',  # ID of the image series
    # 'FileID': 'FileID',  # ID of the image file
    'Admission ID - (0038,0010)': 'AdmissionID',  # ID of the patient's hospital stay
    'Series Number - (0020,0011)': 'SeriesNumber',  # Number of the series in an exam
    'Series Description - (0008,103E)': 'SeriesDescription',  # Description of the image series
    'Image Type - (0008,0008)': 'ImageType',  # Type of the image (PRIMARY, etc.)
    'Derivation Description - (0008,2111)': 'DerivationDescription',  # Description of the image derivation
}, inplace=True)

### Labeled Data CSV

In [ ]:
# Rename columns in df_labels to be more readable
df_labels.rename(columns={
    'number': 'ExamCodeEnd',  # Last 4 digits of the exam code
    'id_number': 'AdmissionID',  # ID of the patient's hospital stay
}, inplace=True)

## Check ArchiMed data to merge with Labeled data

In [ ]:
def analyze_primary_images_distribution(df_archimed):
    """
    Analyzes the distribution of PRIMARY images across exams in the ArchiMed dataset.
    
    Args:
        df_archimed (pd.DataFrame): ArchiMed dataframe containing exam and image information
        
    Returns:
        tuple: (primary_images_per_exam, exams_with_no_primary, exams_with_multiple)
    """
    # Count number of primary images per exam
    primary_images_per_exam = df_archimed[df_archimed['ImageType'].str.contains('PRIMARY', na=False)].groupby('ExamCode').size()

    # Group exams by number of primary images they contain
    exams_distribution = primary_images_per_exam.value_counts().sort_index()
    print(f"{ANSI['G']}Distribution of PRIMARY images per exam:{ANSI['W']}")
    for count, num_exams in exams_distribution.items():
        print(f"  {num_exams} exams have {count} PRIMARY images")

    # Get total number of unique exams
    total_exams = df_archimed['ExamCode'].nunique()

    # Calculate exams with no primary images
    exams_with_no_primary = total_exams - len(primary_images_per_exam)
    exams_with_multiple = len(primary_images_per_exam[primary_images_per_exam > 1])

    print(f"\n{ANSI['G']}Summary:{ANSI['W']}")
    print(f"  {len(primary_images_per_exam)} exams have at least one PRIMARY image")
    print(f"  {exams_with_no_primary} exams have no PRIMARY images")
    print(f"  {exams_with_multiple} exams have multiple PRIMARY images")
    
    return primary_images_per_exam, exams_with_no_primary, exams_with_multiple

# Call the function
primary_images_per_exam, exams_with_no_primary, exams_with_multiple = analyze_primary_images_distribution(df_archimed)

def filter_primary_images(df):
    """
    Filters a dataframe to keep only PRIMARY images.
    
    Args:
        df (pd.DataFrame): DataFrame containing image information with ImageType column
        
    Returns:
        pd.DataFrame: Filtered dataframe containing only PRIMARY images
    """
    df_primary = df[df['ImageType'].str.contains('PRIMARY', na=False)]
    print(f"\n{ANSI['G']}Successfully filtered{ANSI['W']} PRIMARY images from ArchiMed data")
    return df_primary

# Filter ArchiMed data to keep only PRIMARY images
df_archimed_primary = filter_primary_images(df_archimed)


In [ ]:
def print_series_with_multiple_primary(primary_images_per_exam):
    """
    Prints exam codes that have multiple primary images.
    
    Args:
        primary_images_per_exam (pd.Series): Series containing count of primary images per exam
    """
    series_with_multiple = primary_images_per_exam[primary_images_per_exam > 1].sort_values(ascending=False)
    print(f"{ANSI['G']}Series with multiple PRIMARY images:{ANSI['W']}")
    for exam_code, num_primary in series_with_multiple.items():
        print(f"  ExamCode: {exam_code} ({num_primary} primary images)")

# Call the function
print_series_with_multiple_primary(primary_images_per_exam)

## Merge ArchiMed Data with Labeled Data

In [ ]:
def merge_archimed_with_labels(df_archimed, df_labels):
    """
    Merges ArchiMed data with Labeled data on ExamCodeEnd.
    
    Args:
        df_archimed (pd.DataFrame): DataFrame containing ArchiMed data
        df_labels (pd.DataFrame): DataFrame containing labeled data
        
    Returns:
        pd.DataFrame: Merged dataframe with selected columns in specified order
    """
    # Extract last 4 digits from ExamCode in df_archimed and convert to int
    df_archimed['ExamCodeEnd'] = df_archimed['ExamCode'].str[-4:].astype(int)
    
    # Check if AdmissionID exists in both dataframes
    admission_in_archimed = 'AdmissionID' in df_archimed.columns
    admission_in_labels = 'AdmissionID' in df_labels.columns
    
    # Handle duplicate columns for the merge
    if admission_in_archimed and admission_in_labels:
        # Create temporary column names for the merge
        df_archimed = df_archimed.rename(columns={'AdmissionID': 'AdmissionID_archimed'})
        df_labels = df_labels.rename(columns={'AdmissionID': 'AdmissionID_labels'})
    
    # Merge dataframes on ExamCodeEnd
    df_merged = pd.merge(df_archimed, df_labels, on='ExamCodeEnd', how='inner')
    
    # Handle AdmissionID after merge if it existed in both dataframes
    if admission_in_archimed and admission_in_labels:
        # Check if values are identical where both exist
        mask_both_exist = df_merged['AdmissionID_archimed'].notna() & df_merged['AdmissionID_labels'].notna()
        if mask_both_exist.any():
            mismatch = df_merged.loc[mask_both_exist, 'AdmissionID_archimed'] != df_merged.loc[mask_both_exist, 'AdmissionID_labels']
            if mismatch.any():
                print(f"\n{ANSI['Y']}WARNING:{ANSI['W']} AdmissionID values differ between ArchiMed and Labels datasets for {mismatch.sum()} rows")
        
        # Use archimed value if available, otherwise use labels value
        df_merged['AdmissionID'] = df_merged['AdmissionID_archimed'].fillna(df_merged['AdmissionID_labels'])
        
        # Drop temporary columns
        df_merged = df_merged.drop(['AdmissionID_archimed', 'AdmissionID_labels'], axis=1)
    elif admission_in_archimed:
        # AdmissionID only in archimed, no action needed
        pass
    elif admission_in_labels:
        # AdmissionID only in labels, no action needed
        pass
    
    # Get unmerged entries from both dataframes
    df_unmerged_archimed = df_archimed[~df_archimed['ExamCodeEnd'].isin(df_merged['ExamCodeEnd'])]
    df_unmerged_labels = df_labels[~df_labels['ExamCodeEnd'].isin(df_merged['ExamCodeEnd'])]
    
    print(f"\n{ANSI['G']}Successfully merged{ANSI['W']} ArchiMed data with Labeled data")
    print(f"Number of rows in merged dataset: {len(df_merged)}")
    print(f"Number of unmerged rows from ArchiMed: {len(df_unmerged_archimed)}")
    print(f"Number of unmerged rows from Labels: {len(df_unmerged_labels)}")
    
    # Select and reorder columns, checking if each column exists
    columns_order = [
        'ExamCode', 'SeriesNumber', 'SeriesDescription', 'FileID', 'Filename',
        'ImageType', 'DerivationDescription', 'AdmissionID', 'csi_total', 'csi',
        'right_sup', 'left_sup', 'right_mid', 'left_mid', 'right_inf', 'left_inf'
    ]
    
    # Filter out columns that don't exist in the merged dataframe
    available_columns = [col for col in columns_order if col in df_merged.columns]
    
    # Report any missing columns
    missing_columns = set(columns_order) - set(available_columns)
    if missing_columns:
        print(f"\n{ANSI['Y']}WARNING:{ANSI['W']} The following columns are missing from the merged dataframe: {', '.join(missing_columns)}")
    
    return df_merged[available_columns]

# Call the function
df_merged = merge_archimed_with_labels(df_archimed, df_labels)


## Delete Series that are not 'Chest' when there are multiple series

In [ ]:
def remove_non_chest_series(df_merged):
    """
    Removes series that are not 'Chest' or 'LIT' when an exam has multiple series.
    
    Args:
        df_merged (pd.DataFrame): DataFrame containing merged data
        
    Returns:
        tuple: (filtered_df, removed_df) - the filtered DataFrame and the removed rows
    """
    # First, identify ExamCodes with multiple distinct Series
    series_per_exam = df_merged.groupby('ExamCode')['SeriesNumber'].nunique()
    exams_with_multiple = series_per_exam[series_per_exam > 1].index

    # Filter rows where ExamCode has multiple series but SerieDescription is not 'Chest' or 'LIT'
    mask = (
        (df_merged['ExamCode'].isin(exams_with_multiple)) & 
        (~df_merged['SeriesDescription'].str.contains('Chest|LIT', case=False, na=False))
    )

    # Save rows that will be removed
    df_lines_removed = df_merged[mask].copy()

    # Remove these rows
    df_filtered = df_merged[~mask].copy()

    print(f"\n{ANSI['G']}Successfully removed non-Chest series{ANSI['W']} from exams with multiple series")
    print(f"Number of rows removed: {mask.sum()}")
    print(f"Number of rows remaining: {len(df_filtered)}")
    print(f"Removed rows saved in df_lines_removed")
    
    return df_filtered, df_lines_removed

# Call the function
df_merged, df_lines_removed = remove_non_chest_series(df_merged)

In [ ]:
def check_remaining_series_after_removal(df_merged, df_lines_removed):
    """
    Checks that there is still at least 1 serie per exam after removing Series that are not 'Chest'
    
    Args:
        df_merged (pd.DataFrame): DataFrame after filtering
        df_lines_removed (pd.DataFrame): DataFrame containing removed rows
        
    Returns:
        list: List of exam codes that have no remaining series
    """
    # Get unique ExamCodes from removed rows
    removed_exam_codes = df_lines_removed['ExamCode'].unique()

    # Check how many distinct series remain for each of these ExamCodes
    remaining_series = df_merged[df_merged['ExamCode'].isin(removed_exam_codes)].groupby('ExamCode')['SeriesNumber'].nunique()

    # Check if any ExamCode has 0 remaining series
    empty_exams = removed_exam_codes[~np.isin(removed_exam_codes, remaining_series.index)]

    if len(empty_exams) > 0:
        print(f"\n{ANSI['R']}Warning:{ANSI['W']} {len(empty_exams)} exams have no remaining series after removal:")
        print(empty_exams)
    else:
        print(f"\n{ANSI['G']}All exams still have at least one series{ANSI['W']} after removing non-Chest series")
        print("Minimum series per exam:", remaining_series.min())
        print("Maximum series per exam:", remaining_series.max())
        print("Average series per exam: {:.2f}".format(remaining_series.mean()))
    
    return empty_exams

# Call the function
empty_exams = check_remaining_series_after_removal(df_merged, df_lines_removed)

In [ ]:
def display_series_distribution(df):
    """
    Displays the distribution of series per exam and images per series.
    
    Args:
        df (pd.DataFrame): DataFrame containing exam data with ExamCode and SeriesNumber columns
    """
    # Count number of series per exam
    series_per_exam = df.groupby('ExamCode')['SeriesNumber'].nunique()
    
    # Get distribution of series per exam
    distribution = series_per_exam.value_counts().sort_index()
    
    print(f"\n{ANSI['G']}Distribution of series per exam:{ANSI['W']}")
    for num_series, count in distribution.items():
        print(f"{count} exams have {num_series} series")
    
    # Count number of images per series
    images_per_series = df.groupby(['ExamCode', 'SeriesNumber'])['SOPInstanceUID'].count()
    
    # Get distribution of images per series
    img_distribution = images_per_series.value_counts().sort_index()
    
    print(f"\n{ANSI['G']}Distribution of images per series:{ANSI['W']}")
    for num_images, count in img_distribution.items():
        print(f"{count} series have {num_images} images")
    
    print(f"\n{ANSI['G']}Total exams:{ANSI['W']} {len(series_per_exam)}")
    print(f"{ANSI['G']}Total images:{ANSI['W']} {df['SOPInstanceUID'].nunique()}")
    print(f"{ANSI['G']}Average series per exam:{ANSI['W']} {series_per_exam.mean():.2f}")

# Call the function
display_series_distribution(df_merged)